In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import ROOT
from scipy.integrate import dblquad

sys.path.append('/Users/alexanderantonakis/Desktop/Software/CEvNS_RATES/Classes')
from snsgen import SNSGen

print("Finished importing")

# Initialize the SNS Generator for He: 1 Ton, 20 m Baseline

In [ ]:
# initialize the SNS generator with the material of choice

mdet = 1000 #kg
L = 20. # meters
# Expected POT/year
pot = 2.1*(10**23)
#material = "Ar40"
material = "He"

gen = SNSGen(mdet, L, pot, material)

print("Set up the SNS generator for our config")

# Neutrino Flux Spectral Functions

In [ ]:
Es = np.arange(0, 53, 0.001)
s_nue = gen.spectral_nue(Es)
s_numubar = gen.spectral_numubar(Es)
s_numu = gen.spectral_numu(Es)
plt.plot(Es, s_nue, label=r"$\nu_{e}$")
plt.plot(Es, s_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.plot(Es, s_numu, label=r"$\nu_{\mu}$")
plt.legend(fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Spectral Shape [Arb.]", fontsize=14)
plt.show()

# POT Normalized Flux

In [ ]:
f_nue = gen.flux_nue(Es)
f_numubar = gen.flux_numubar(Es)
f_numu = gen.flux_numu(Es)
plt.scatter([], [], label="Detector Baseline: " + str(gen.L)+" m")
plt.plot(Es, f_nue, label=r"$\nu_{e}$")
plt.plot(Es, f_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.plot(Es, f_numu, label=r"$\nu_{\mu}$")
plt.legend(fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Neutrino Flux [" + r"$\nu$"+"/"+"$m^2$"+"/POT]", fontsize=14)
plt.yscale("log")
plt.show()

# Helm Form Factor

In [ ]:
Es = np.arange(0.001, 0.14+0.001, 0.001)
qs = np.sqrt(2*gen.M*Es)
fs = gen.FH(Es)**2
plt.plot(qs, fs)
plt.xlabel("Momentum Transfer [MeV]", fontsize=14)
plt.ylabel(r"Helm Form Factor Squared", fontsize=14)
plt.title("Material: Helium")
plt.show()

# Cross Section

In [ ]:
Enus = np.linspace(0.00001, 53, 1000)
sigmas = gen.Sigma(Enus)


plt.plot(Enus, sigmas*(100**2)*(10**38))
plt.xlim([5, 53])
plt.ylim([10**(-5), 10**2])
plt.yscale("log")
plt.ylabel("Cross-section ("+r"$10^{-38} cm^2$"+")", fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.show()

# Event Rate as a function of the Recoil Energy

In [ ]:
Ers = np.arange(0.001, 0.15, 0.001)
plt.plot(Ers*1000, gen.N_Er_numubar(Ers), label=r"$\bar{\nu}_{\mu}$")
plt.plot(Ers*1000, gen.N_Er_nue(Ers), label=r"$\nu_{e}$")
plt.plot(Ers*1000, gen.N_Er_numu(Ers), label=r"$\nu_{\mu}$")
plt.ylabel("Events/POT/keV", fontsize=14)
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.legend(fontsize=14)
plt.xlim([1, 140])
#plt.ylim([0, 350])
plt.show()

In [ ]:
Ers = np.arange(0.001, 0.15, 0.001)
plt.plot(Ers*1000, gen.pot*gen.N_Er_numubar(Ers), c="r", label=r"$\bar{\nu}_{\mu}$")
plt.plot(Ers*1000, gen.pot*gen.N_Er_nue(Ers), c="g", label=r"$\nu_{e}$")
plt.plot(Ers*1000, gen.pot*gen.N_Er_numu(Ers), c="b", label=r"$\nu_{\mu}$")
plt.ylabel("Events/year/keV", fontsize=14)
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.legend(fontsize=14)
plt.xlim([1, 160])
#plt.ylim([0, 315])
plt.show()

# Recoil Spectrum

In [ ]:
def N(Er1, Er2, cos1, cos2):
    def f(y, x):
        return 4*math.pi*gen.dN_dEr_dcos_muon(y, x)
        
    v = dblquad(f, cos1, cos2, Er1, Er2)
    #print(v)
    return v[0]

print(N(1/1000, 10/1000, 0.2, 0.6)*gen.pot)

In [ ]:
Ermin = 1
Ermax = 1500

nbinsx = Ermax-Ermin

cosmin = 0.01
cosmax = 1

coswidth = 0.01
nbinsy = int((cosmax - cosmin)/coswidth)


h = ROOT.TH2D("N_2D", "", nbinsx, Ermin, Ermax, nbinsy, cosmin, cosmax)

print("Start Loop Over", nbinsx*nbinsy, " Cells")
count = 0
for i in range(1, h.GetNbinsX()+1):
    for j in range(h.GetNbinsY()+1):
        Er1 = h.GetXaxis().GetBinLowEdge(i)
        Er2 = Er1 + h.GetXaxis().GetBinWidth(i)
        cos1 = h.GetYaxis().GetBinLowEdge(j)
        cos2 = cos1 + h.GetYaxis().GetBinWidth(j)
        v = N(Er1/1000, Er2/1000, cos1, cos2)*gen.pot
        h.SetBinContent(i, j, v)
        count += 1
        if count % 10000 == 0:
            print("Cell", count)

c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
h.Draw("Colz")
c.Draw()

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
c.SetLogx()
h.SetStats(0)
h.GetXaxis().SetTitle("Nuclear Recoil Energy [keV]")
h.GetYaxis().SetTitle("cos#theta_{r}")
h.GetZaxis().SetTitle("Interactions/Year")
h.GetXaxis().SetTitleOffset(1.5)
h.GetZaxis().SetTitleOffset(0.7)
h.Draw("Colz")
c.Draw()

In [ ]:
rfile = ROOT.TFile.Open("../Main/he_spectrum.root", "READ")
h_multi = rfile.Get("N_2D")
h_multi.SetDirectory(0)
rfile.Close()

# Check that the Multiprocessing version matches from Main

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
c.SetLogx()
h_multi.SetStats(0)
h_multi.GetXaxis().SetTitle("Nuclear Recoil Energy [keV]")
h_multi.GetYaxis().SetTitle("cos#theta_{r}")
h_multi.GetZaxis().SetTitle("Interactions/Year")
h_multi.GetXaxis().SetTitleOffset(1.5)
h_multi.GetZaxis().SetTitleOffset(0.7)
h_multi.Draw("Colz")
c.Draw()

In [ ]:
rfile = ROOT.TFile.Open("../Main/Ar_spectrum.root", "READ")
h_multi_Ar = rfile.Get("N_2D")
h_multi_Ar.SetDirectory(0)
rfile.Close()